## Dataset Generation
You can do dynamic generation on the go, but it usually takes around 1.5 hours to go through the dataset. I prefer to use statically generated dataset because it allows to iterate throught the dataset in am matter of 4 minutes. Frame size in the number of actions you want to be in the state (the number of movies rated). I have chosen it to be 10, if you want another number you will have to generate it yourself. Link to the google drive is in the readme.md


## This is a mandatory thing to do, you can download the existing dataset here: [Link](https://drive.google.com/open?id=1pPf-7AmUVceVfgfmKEJ6ireEDKEJHw-7)



### Things you will need to download:
- [Movie embeddings](https://drive.google.com/open?id=1kTyu05ZmtP2MA33J5hWdX8OyUYEDW4iI) or you can generate them yourself

## Dataset Genaration

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import json
import pickle
import h5py

## Settings here!
frame_size = 10
batch_size = 1000
## End settings

csv_ratings = pd.read_csv('../../data/ml-20m/ratings.csv')
ratings = h5py.File("../../data/test/test.hdf5", "r")

# credits: KnightofK9
users = csv_ratings.groupby(["userId"]).size()
users = users[users >= frame_size + 1]

del csv_ratings

In [2]:
import h5py
import numpy as np
import torch
import pickle

h5 = h5py.File("../../data/test/test_rolling.hdf5", "w")
idx = 0

In [3]:
ds_ratings = h5.create_dataset("ratings", (0, 11), maxshape=(None, 256), dtype='int8', chunks=True, compression="lzf")
ds_movies = h5.create_dataset("movies", (0, 11), maxshape=(None, 256), dtype='int32',  chunks=True, compression="lzf")
ds_done = h5.create_dataset("done", (0, ), maxshape=(None,), dtype='?', chunks=True,  compression="lzf")

In [35]:
user_bar = tqdm(total=len(users))
batch = []

g_idx = 0
batch = []
for user in users.index:
    user_bar.update(1)
    group = ratings['u' + str(user)]
    movies = group['movies'][:]
    rates = group['ratings'][:]
    size = len(movies) - frame_size
    for idx in range(0, size):
        if np.random.rand() < 0.8:  # intake percents
            continue
        batch.append([movies[idx:frame_size + idx + 1],
                      movies[idx:frame_size + idx + 1],
                      idx + 1 == size])
        if len(batch) % batch_size == 0:
            
            [i.resize([len(batch) + i.shape[0], i.shape[1]]) for i in [ds_ratings, ds_movies]]
            [i.resize([len(batch) + i.shape[0]]) for i in [ds_done]]
            
            for i in batch:
                movies_, ratings_, done_ = i

                movies_ = np.array(movies_)
                ratings_ = np.array(ratings_)
                done_ = np.array(done_)
                ds_movies[g_idx] = movies_.astype('int32')
                ds_ratings[g_idx] = ratings_.astype('int8')
                ds_done[g_idx] = done_.astype('?')
                
                g_idx+=1

            batch = []
        

KeyboardInterrupt: 

## Test:

In [ ]:
import torch
import pickle
from tqdm import tqdm_notebook as tqdm
import h5py
import numpy as np

movie_ref = pickle.load(open('../data/infos_pca128.pytorch', 'rb'))

f = h5py.File("../../data/test/test_rolling.hdf5", "r")

cuda = torch.device('cuda')

def prepare_batch(*args):
    args = [torch.tensor(np.array(arg).astype(np.float)).to(cuda) for arg in args]
    return args


batch_size = 5000
losses = []
T_losses = []


for i in tqdm(range(f['done'].shape[0] // batch_size)):
    movies, ratings, done = [f[key][i*batch_size:(i+1)*batch_size] for key in
             ['movies', 'ratings', 'done']]
    
    movies, ratings, done = [torch.tensor(i.astype('float32')) for i in [movies, ratings, done]]
    movies_tensor = torch.stack([torch.stack([movie_ref[int(i)] for i in ts]) for ts in movies])
    
    state = torch.cat([movies_tensor[:, :-1, :].view(state.size(0), -1),
                       ratings[:, :-1]], 1)
    next_state = torch.cat([movies_tensor[:, 1:, :].view(state.size(0), -1),
                            ratings[:, 1:]], 1)
    action = movies_tensor[:, -1]
    reward = ratings[:, -1]
